In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## The forward and backward passes

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=4960)

In [2]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return ... # ??? what's normalize given x is data, m is mean, and s is std.

In [3]:
x_train,y_train,x_valid,y_valid = get_data()

In [4]:
train_mean, train_std = ... # ??? how to get mean and std from fastai dataset class
train_mean, train_std # (tensor(0.1304), tensor(0.3073))

(tensor(0.1304), tensor(0.3073))

In [5]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std) 
#??? think: why we use mean and std only from train and not test?

In [6]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std # (tensor(0.0001), tensor(1.))

(tensor(0.0001), tensor(1.))

In [7]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [8]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [9]:
n,m = x_train.shape
c = y_train.max()+1 #bcz of 0
n,m,c # number of data/sample, number of columns/features, number of class/y

(50000, 784, tensor(10))

## Foundations version

### Basic architecture

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=5128)

In [1]:
# num hidden
nh = 50
c = 1 # c shall be 10 but lets assume it is 1 for now -> in order to demonstrate squeeze later

[Tinker practice](https://course.fast.ai/videos/?lesson=8&t=5255)

In [11]:
# ??? simplified kaiming init / he init, assuming first layer m -> nh, second layer nh -> 1
w1 = ...
b1 = ...
w2 = ...
b2 = ...

In [12]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [13]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0057), tensor(0.9924))

In [14]:
def lin(x, w, b): return x@w + b

In [15]:
t = lin(x_valid, w1, b1)

In [16]:
#...so should this, because we used kaiming init, which is designed to do this
t.mean(),t.std()

(tensor(-0.0718), tensor(0.9930))

In [17]:
def relu(x): return ... # ??? find a torch function to conduct relu

In [18]:
t = relu(lin(x_valid, w1, b1))

In [19]:
#...actually it really should be this!
t.mean(),t.std()

(tensor(0.3585), tensor(0.5668))

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=5128)

In [20]:
# kaiming init / he init for relu
w1 = ... # ??? what shall w1 be with kaiming init?

In [21]:
w1.mean(),w1.std()

(tensor(0.0003), tensor(0.0503))

In [22]:
t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()

(tensor(0.5273), tensor(0.8173))

In [23]:
#export
from torch.nn import init

In [24]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out') #??? what is mode fan out
t = relu(lin(x_valid, w1, b1))

In [25]:
init.kaiming_normal_??

In [26]:
w1.mean(),w1.std()

(tensor(2.3772e-05), tensor(0.0503))

In [27]:
t.mean(),t.std()

(tensor(0.4501), tensor(0.7462))

In [28]:
w1.shape

torch.Size([784, 50])

In [29]:
import torch.nn

In [30]:
torch.nn.Linear(m,nh).weight.shape

torch.Size([50, 784])

In [31]:
torch.nn.Linear.forward??

In [32]:
torch.nn.functional.linear??

In [33]:
torch.nn.Conv2d??

In [34]:
torch.nn.modules.conv._ConvNd.reset_parameters??

In [35]:
# what if...?
def relu(x): return x.clamp_min(0.) - 0.5

In [36]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(0.0482), tensor(0.8306))

In [37]:
def model(xb):
    # use the components obviou (def lin, and def relu, w1, w2, b1, b2), 
    # build a simple linear-relu-linear model
    l1 = ...
    l2 = ...
    l3 = ...
    return l3

In [38]:
%timeit -n 10 _=model(x_valid)
# expect around 7.3 ms ± 1.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)

7.3 ms ± 1.16 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [39]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

### Loss function: MSE

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=6372)

In [40]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [41]:
#export
def mse(output, targ): return ... #??? how to compute mse in code? and why do we need to use squeeze()

In [42]:
y_train,y_valid = y_train.float(),y_valid.float() #??? does this line matter?

In [43]:
preds = model(x_train)

In [44]:
preds.shape

torch.Size([50000, 1])

In [45]:
mse(preds, y_train) # around 30 shall be fine.

tensor(27.8080)

### Gradients and backward pass

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=6493)

In [46]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = ... # ??? what's the gradient of mse (notice its pre-determined)

# test:
dummy_target, dummy_output = torch.randn(10), torch.randn(10)
mse_grad(dummy_output, dummy_target)
dummy_output.g.shape # [10, 1]

In [ ]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = ... # ???
    w.g = ...
    b.g = ...

# test
dummy_input = torch.randn(10,30)
dummy_w = torch.randn(20, 1)
dummy_b = torch.randn(10)
lin_grad(dummy_input, dummy_output, dummy_w, dummy_b)
print(dummy_input.g.shape) # [10,20]
print(dummy_w.g.shape) # [30,1]
print(dummy_b.g.shape) # 1

In [47]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = ... # ??? what's the gradient of relu
    
# test:
relu_grad(dummy_input, dummy_output)
dummy_input.g.shape # [10, 30]

In [49]:
def forward_and_backward(inp, targ):
    #??? simple forward pass: linear-relu-linear, assume we have w1, b1, w2, b2 defined
    l1 = ...
    l2 = ...
    out = ...
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass: ??? 
    mse_grad(...)
    lin_grad(...)
    relu_grad(...)
    lin_grad(...)

In [ ]:
% time forward_and_backward(x_train, y_train)
# shall take around CPU times: user 7.85 s, sys: 9.48 s, total: 17.3 s
# if it takes a super long time, compare your grad functions to answers

In [50]:

# Test:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

# We cheat a little bit and use PyTorch autograd to check our results.

xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

# here's how you shall train in pytorch.
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

loss = forward(xt2, y_train)

loss.backward()

test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

## Refactor model

### Layers as classes

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=7112)

In [57]:
# ??? you have done all of them, now refactor.
class Relu():
    def __call__(self, inp):
        self.inp = ...
        self.out = ...
        return self.out
    
    def backward(self): self.inp.g = ...

In [58]:
# ??? you have done all of them, now refactor.
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = ...
        return self.out
    
    def backward(self):
        self.inp.g = ...
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = ...
        self.b.g = ...

In [59]:
# ??? you have done all of them, now refactor.
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = ...
        return self.out
    
    def backward(self):
        self.inp.g = ...

In [60]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [61]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [62]:
%time loss = model(x_train, y_train)
# expect around CPU times: user 124 ms, sys: 4.06 ms, total: 128 ms

CPU times: user 124 ms, sys: 4.06 ms, total: 128 ms
Wall time: 33.7 ms


In [63]:
%time model.backward()
# expect around CPU times: user 5.1 s, sys: 2.55 s, total: 7.65 s

CPU times: user 5.1 s, sys: 2.55 s, total: 7.65 s
Wall time: 7.03 s


In [64]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Module.forward()
with einsum

In [65]:
# not a big fan of einsum...so I just skipped this part. feel free to remove some lines and do it yourself.
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        self.b.g = out.g.sum(0)

class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [71]:
%time loss = model(x_train, y_train)

CPU times: user 129 ms, sys: 4.11 ms, total: 133 ms
Wall time: 34.5 ms


In [72]:
%time model.backward()

CPU times: user 489 ms, sys: 51.9 ms, total: 541 ms
Wall time: 137 ms


In [73]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Without einsum

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=7484)

In [74]:
class Lin(Module):
    # ??? fill them in :D
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return ...
    
    def bwd(self, out, inp):
        inp.g = ...
        self.w.g = ...
        self.b.g = ...

In [75]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [76]:
%time loss = model(x_train, y_train)
# expect around CPU times: user 119 ms, sys: 2.65 ms, total: 121 ms

CPU times: user 119 ms, sys: 2.65 ms, total: 121 ms
Wall time: 31.9 ms


In [77]:
%time model.backward()
# expect around CPU times: user 468 ms, sys: 51.2 ms, total: 520 ms

CPU times: user 468 ms, sys: 51.2 ms, total: 520 ms
Wall time: 130 ms


In [78]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### nn.Linear and nn.Module

In [79]:
#export
from torch import nn

In [80]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(...), nn.ReLU(), nn.Linear(...)] #??? easy ones
        self.loss = mse
        
    def __call__(self, x, targ):
        for layer in self.layers: x = layer(x)
        return self.loss(x.squeeze(), targ)

In [81]:
model = Model(m, nh, 1)

In [82]:
%time loss = model(x_train, y_train)
# expect around CPU times: user 102 ms, sys: 4.13 ms, total: 106 ms

CPU times: user 102 ms, sys: 4.13 ms, total: 106 ms
Wall time: 33.2 ms


In [83]:
%time loss.backward()
# expect around CPU times: user 308 ms, sys: 8.94 ms, total: 317 ms

CPU times: user 308 ms, sys: 8.94 ms, total: 317 ms
Wall time: 48.3 ms


## Export

In [84]:
!./notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to exp/nb_02.py
